# bRAG: Basic (naive) RAG Implementation

This notebook demonstrates a complete implementation of a basic RAG system that enables question-answering over PDF documents. 
The implementation is model, database, and document loader agnostic, though it's currently configured with:
- LLM: OpenAI GPT-3.5-turbo
- Vector Database: Pinecone
- Document Loader: PyPDFLoader

The system combines several key components:
1. Document Loading: Loads PDF documents (extensible to other document types)
2. Text Processing: Splits documents into manageable chunks
3. Vector Operations:
   - Embeds text using OpenAI's embedding model
   - Stores vectors in Pinecone vector database
4. Retrieval System: Implements efficient document retrieval
5. LLM Integration: Uses OpenAI's GPT model for generating responses

All components can be swapped out for alternatives (e.g., different LLMs, vector stores, or document loaders) 
while maintaining the same overall architecture.

This implementation serves as a foundation for building more complex RAG applications
and can be customized based on specific use cases.

----------------------------------------

## Pre-requisites (optional but recommended)

### Only do the first step if you have never created a virtual environment for this repository. Otherwise, make sure that the Python Kernel that you selected is from your `venv/` folder.

In [ ]:
# Create virtual environment
! python -m venv venv

In [ ]:
# Activate virtual Python environment
! source venv/bin/activate

In [ ]:
# If your Python is not from your venv path, ensure that your IDE's kernel selection (on the top right corner) is set to the correct path 
# (your path output should contain "...venv/bin/python")

! which python

/Users/taha/Desktop/bRAGAI/code/gh/bRAG-langchain/venv/bin/python


In [ ]:
# Install all packages
! pip install -r requirements.txt --quiet

## Environment

`(1) Packages`

In [1]:
import os
from dotenv import load_dotenv

# Load all environment variables from .env file
load_dotenv()

# Access the environment variables
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

## LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

## Pinecone Vector Database
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_host = os.getenv('PINECONE_API_HOST')
index_name = os.getenv('PINECONE_INDEX_NAME')


`(2) LangSmith`

https://docs.smith.langchain.com/

In [2]:
os.environ['LANGCHAIN_TRACING_V2'] = langchain_tracing_v2
os.environ['LANGCHAIN_ENDPOINT'] = langchain_endpoint
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

`(3) API Keys`

In [3]:
os.environ['OPENAI_API_KEY'] = openai_api_key
openai_model = "gpt-3.5-turbo"

#Pinecone keys
os.environ['PINECONE_API_KEY'] = pinecone_api_key
os.environ['PINECONE_API_HOST'] = pinecone_api_host
os.environ['PINECONE_INDEX_NAME'] = index_name

`(4) Pinecone Init`

In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index(os.environ['PINECONE_INDEX_NAME'])

/Users/taha/Projects/BragAI/code/gh/bRAG-langchain/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Full RAG App (Basic)

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate

#### INDEXING ####

pdf_file_path = "test/langchain_turing.pdf"
loader = PyPDFLoader(pdf_file_path)

docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = PineconeVectorStore.from_documents(
    documents=splits, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"), 
    index_name=index_name
)

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name=openai_model, temperature=0.1)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
# Question
from pprint import pprint

pprint(rag_chain.invoke("What is this document about?"))

('This document is about LangChain, its modular design, security implications, '
 'capabilities, challenges, and its potential in advancing NLP applications. '
 'It also serves as a resource for developers and researchers interested in '
 'LLM application development, providing insights into how to effectively '
 'harness LangChain to build innovative and secure LLM-powered applications.')
